# Pickle

In [1]:
import os
import pickle
import sys

from typing import Dict

import dill

## Saving Something "Complex"

### Func

In [2]:
def hello_world():
    print("Hello world!")

In [3]:
hello_world()

Hello world!


In [4]:
pickle.dumps(hello_world)

b'\x80\x04\x95\x1c\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x0bhello_world\x94\x93\x94.'

In [5]:
with open('hello.bin', 'wb') as f:
    f.write(pickle.dumps(hello_world))

In [6]:
with open('hello.bin', 'rb') as f:
    data = f.read()

In [7]:
data

b'\x80\x04\x95\x1c\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x0bhello_world\x94\x93\x94.'

In [8]:
func = pickle.loads(data)

In [9]:
func()

Hello world!


In [10]:
del hello_world

### Class

In [11]:
class Somebody:
    def __init__(self):
        self.age = 8
        self.name = 'Kevin'
    
    def say_hello(self):
        return 'Hello!'

In [12]:
pickle.dumps(Somebody)

b'\x80\x04\x95\x19\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x08Somebody\x94\x93\x94.'

In [13]:
obj = Somebody()

In [14]:
pickle.dumps(obj)

b'\x80\x04\x958\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x08Somebody\x94\x93\x94)\x81\x94}\x94(\x8c\x03age\x94K\x08\x8c\x04name\x94\x8c\x05Kevin\x94ub.'

In [15]:
with open('object.pkl', 'wb') as file:
    file.write(pickle.dumps(obj))

In [16]:
restored = pickle.loads(
    open('object.pkl', 'rb').read()
)

In [17]:
restored.age, restored.name

(8, 'Kevin')

## Where is Kevin?

Уберём класс из namespace-а ноутбука...

In [18]:
del Somebody

И снова попытаемся восстановить объект класса:

In [19]:
try:
    restored = pickle.loads(
        open('object.pkl', 'rb').read()
    )
except AttributeError as error:
    print(f'Failed to restore! Error: "{error}".')

Failed to restore! Error: "Can't get attribute 'Somebody' on <module '__main__'>".


Дело в том, что `pickle` не дампит "собственно сам класс" — он лишь сохраняет информацию о том, *где можно достать* класс. И если этого "где" нет (или в этом "где" уже нет такого класса), то и восстановления — нет `¯\_(ツ)_/¯`

Попробуем теперь восстановить некоторый заранее сохранённый `numpy` массив. (Библиотека `numpy` при этом в ноутбуке не импортировалась.)

In [20]:
restored_array = pickle.loads(
    open('array.pkl', 'rb').read()
)

In [21]:
restored_array

array([1, 2, 3])

In [22]:
type(restored_array)

numpy.ndarray

In [23]:
try:
    numpy
except NameError:
    print('No numpy!')

No numpy!


Таким образом, получилось восстановить массив, но при этом библиотека `numpy` как была недоступна в ноутбуке, так и осталась. Дело в том, что в процессе загрузки объекта `pickle` и находит, и импортирует `numpy`, но [импортирует "локально"](https://github.com/python/cpython/blob/a365dd64c2a1f0d142540d5031003f24986f489f/Lib/pickle.py#L1067), так, что в ноутбуке новых импортов не возникает.
Однако всё равно можно понять, что `numpy` "потрогали" — с помощью словаря модулей `sys.modules`, которые были импортированы в процессе работы программы.

In [24]:
'numpy' in sys.modules

True

In [25]:
sys.modules['numpy']

<module 'numpy' from '/home/alvant/lib/miniconda3/envs/nis/lib/python3.8/site-packages/numpy/__init__.py'>

## Selective Save

Не всё можно сдампить с помощью пикла.

In [26]:
class Somebody:
    def __init__(self):
        self.age = 8
        self.name = 'Kevin'
        self.say_hello = lambda: 'Hello!'

In [27]:
obj = Somebody()

In [28]:
obj.say_hello()

'Hello!'

In [29]:
try:
    pickle.dumps(obj)
except AttributeError as error:
    print(f'Failed to dump! Error: "{error}".')

Failed to dump! Error: "Can't pickle local object 'Somebody.__init__.<locals>.<lambda>'".


В таких случаях надо "руками" определять, что должно быть сохранено пиклом.
По умолчанию пикл пытается сохранить `__dict__` объекта:

In [30]:
obj.__dict__

{'age': 8,
 'name': 'Kevin',
 'say_hello': <function __main__.Somebody.__init__.<locals>.<lambda>()>}

Поэтому уберём из словаря с характеристиками объекта (далее назван как `state`) всё лишнее:

In [31]:
class Somebody:
    def __init__(self):
        self.age = 8
        self.name = 'Kevin'
        self.say_hello = lambda: 'Hello!'
    
    def __getstate__(self) -> Dict:
        state = self.__dict__.copy()
        
        del state['say_hello']
        
        return state

In [32]:
obj = Somebody()

pickle.dumps(obj)

b'\x80\x04\x958\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x08Somebody\x94\x93\x94)\x81\x94}\x94(\x8c\x03age\x94K\x08\x8c\x04name\x94\x8c\x05Kevin\x94ub.'

In [33]:
restored = pickle.loads(
    pickle.dumps(obj)
)

In [34]:
try:
    restored.say_hello
except AttributeError as error:
    print(f'Where is `say_hello`? Failed to restore properly!'
          f' Error: "{error}"')

Where is `say_hello`? Failed to restore properly! Error: "'Somebody' object has no attribute 'say_hello'"


"Убрать" недостаточно — надо ещё руками же и "вернуть" потом на место:

In [35]:
class Somebody:
    def __init__(self):
        self.age = 8
        self.name = 'Kevin'
        self.say_hello = lambda: 'Hello!'
    
    def __getstate__(self) -> Dict:
        state = self.__dict__.copy()
        
        del state['say_hello']
        
        return state
    
    def __setstate__(self, state):
        self.__dict__ = state
        self.say_hello = lambda: 'Bye!'

In [36]:
obj = Somebody()

pickle.dumps(obj)

b'\x80\x04\x958\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x08Somebody\x94\x93\x94)\x81\x94}\x94(\x8c\x03age\x94K\x08\x8c\x04name\x94\x8c\x05Kevin\x94ub.'

In [37]:
restored = pickle.loads(
    pickle.dumps(obj)
)

In [38]:
restored.say_hello()

'Bye!'